In [1]:
from readers import lenta_reader, ria_reader_with_date_approx
import tqdm
from transformers import BertTokenizer, EncoderDecoderModel, logging

import pandas as pd
import numpy as np
import json
import random
from sklearn.metrics.pairwise import cosine_distances
from utils.clustering_utils import get_text_to_vector_func
import multiprocessing as mp
import os
from readers import ria_reader, tg_reader, lenta_reader

In [2]:
import json

def reader(path):
    with open(path, 'r') as f:
        for line in f:
            yield json.loads(line.strip())

In [3]:
train_file = '../../datasets/'

lenta_records = [r for r in tqdm.tqdm(lenta_reader(os.path.join(train_file, 'lenta/lenta-ru-news.train.csv')))]
lenta_records.extend(
    [r for r in tqdm.tqdm(lenta_reader(os.path.join(train_file, 'lenta/lenta-ru-news.val.csv')))]
)

ria_records = [r for r in tqdm.tqdm(ria_reader(os.path.join(train_file, 'ria/ria.shuffled.train.json')))]
ria_records.extend(
    [r for r in tqdm.tqdm(ria_reader(os.path.join(train_file, 'ria/ria.shuffled.val.json')))]
)

597175it [00:38, 15344.05it/s]
75971it [00:03, 19004.02it/s]
858741it [18:30, 773.29it/s] 
47399it [01:09, 684.83it/s] 


In [4]:
records = [r for r in tqdm.tqdm(reader('../../datasets/full_lenta_ria.test.jsonl'))]

filter_lenta = [
    {'text': r['lenta_text'], 'title': r['lenta_title'], 'agency': 'lenta.ru', 'date': r['lenta_date']} 
    for r in records
]

filter_ria = [
    {'text': r['ria_text'], 'title': r['ria_title'], 'agency': 'РИА Новости', 'date': r['lenta_date']} 
    for r in records
]

2000it [00:00, 2560.60it/s]


In [5]:
lenta_filter_titles = set(x['title'] for x in filter_lenta)
ria_filter_titles = set(x['title'] for x in filter_ria)

In [6]:
print(len(lenta_records))
lenta_records = [r for r in lenta_records if r['title'] not in lenta_filter_titles]
len(lenta_records)

673146


671329

In [8]:
len([r for r in lenta_records if r['date'][:4] in ['2010', '2011', '2012', '2013', '2014']])

192738

In [7]:
print(len(ria_records))
ria_records = [r for r in ria_records if r['title'] not in ria_filter_titles]
len(ria_records)

906140


904216

In [ ]:
random.shuffle(ria_records)

all_records = [r for r in lenta_records if r['date'][:4] in ['2010', '2011', '2012', '2013', '2014']] + \
    ria_records[:300000]

random.shuffle(all_records)

In [ ]:
print("Building datasets...")

full_dataset = GenTitleDataset(
    all_records, tokenizer,
    max_tokens_text=max_tokens_text, max_tokens_title=max_tokens_title
)

train_size = int(0.99 * len(full_dataset))
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset,
                                                           [train_size, len(full_dataset) - train_size])

### Accumulation

In [25]:
base_dir = '/home/aobuhtijarov/datasets/'

with open(os.path.join(base_dir, 'full_lenta_ria_dataset.jsonl'), 'w', encoding='utf-8') as f:
    for el in os.listdir(base_dir):
        if not el.startswith('lenta_ria_'):
            continue
        
        with open(os.path.join(base_dir, el), 'r') as fin:
            data = [json.loads(x.strip()) for x in fin.readlines()]
            
        for x in data:
            json.dump(x, f)
            f.write('\n')


### Train / test split

In [27]:
base_dir = '/home/aobuhtijarov/datasets/'

with open(os.path.join(base_dir, 'full_lenta_ria_dataset.jsonl'), 'r', encoding='utf-8') as fin:
    data = [json.loads(x.strip()) for x in fin.readlines()]
    
random.shuffle(data)

with open(os.path.join(base_dir, 'full_lenta_ria.test.jsonl'), 'w', encoding='utf-8') as fout:
    for x in data[:2000]:
        json.dump(x, fout)
        fout.write('\n')

with open(os.path.join(base_dir, 'full_lenta_ria.train.jsonl'), 'w', encoding='utf-8') as fout:
    for x in data[2000:]:
        json.dump(x, fout)
        fout.write('\n')

In [28]:
len(data)

77362

In [29]:
data[0]

{'lenta_text': 'президент россии дмитрий медведев поздравил спикера польского парламента бронислава коморовского с победой на президентских выборах в польше, сообщает риа новости. второй тур досрочных президентских выборов прошел в польше 4 июля. избирательные участки закрылись в 20:00 по местному времени (22:00 по москве). официальные итоги голосования подведут позднее в понедельник, 5 июля. по итогам подсчета примерно 95 процентов бюллетеней, коморовский набирает более 52 процентов голосов, его противник ярослав качиньский - около 47 процентов. 10 апреля 2010 года в авиакатастрофе под смоленском погиб лех качиньский, возглавлявший польшу с 2005 года. в стране были объявлены досрочные президентские выборы. их первый тур состоялся 20 июня. ни один из претендентов не смог преодолеть отметку в 50 процентов голосов, в связи с чем был назначен второй тур голосования.',
 'lenta_title': 'медведев поздравил коморовского с победой на выборах',
 'lenta_date': '2010-07-05 10:00',
 'ria_text': ' 

In [23]:
!cat ../../datasets/lenta_ria_* | wc -l

77362


In [13]:
with open('../../datasets/lenta_ria.jsonl', 'r') as f:
    data = [json.loads(x.strip()) for x in f.readlines()]

In [14]:
len(data)

14318

In [16]:
data[6]

{'lenta_text': 'полиция канады расследует смерть девятерых человек, включая двух детей, тела которых были обнаружены в трех разных местах в городе эдмонтон (провинция альберта). как сообщает телеканал cbc, один из погибших — мужчина, который, предположительно, совершил преступления, а затем покончил жизнь самоубийством. первый вызов на пульт дежурного поступил в понедельник, 29 декабря, около 18:53 (08:53 по московскому времени). полицейские, прибывшие в один из домов, расположенный на юге города, обнаружили тело женщины с огнестрельными ранениями. позднее, как рассказал начальник полиции города род кнехт (rod knecht), сотрудники правоохранительных органов получили вызов из северной части города. войдя в дом, они обнаружили тела семи человек — трех женщин, двух мужчин и двух детей (мальчика и девочки). они также были застрелены. после этого полицейские получили информацию, что неизвестный мужчина покончил с собой в ресторане в городе форт-саскачеван, расположенном в 25 километрах от эд

In [19]:
data[random.randint(0, len(data)-1)]

{'lenta_text': 'департамент градостроительной политики москвы выбрал шесть цветов, в которые можно будет оформлять непрозрачные ограждения на строительных площадках. об этом говорится в сообщении ведомства. заборы на столичных стройках теперь будут только темно-серого, светло-серого, коричнево-серого, песочно-серого, темно-алюминиевого или темно-зеленого оттенков. выполнять заграждения предлагается из поликарбоната, металлического листа или сайдинга. ограды будут классифицироваться в зависимости от места использования. в 2011 году мэр москвы сергей собянин распорядился заменить широко распространенные сплошные бетонные заборы на решетчатые и сетчатые. предполагалось, что "прозрачные" ограждения позволят горожанам и контролирующим органам наблюдать, какие именно работы ведутся на площадках. сообщалось, что демонтажу и замене подлежат 80 процентов из 3500 заборов. в 2013 году власти решили частично вернуться к непрозрачным ограждениям, поскольку в ряде случаев это необходимо для соблюден

In [1]:
from readers import lenta_reader, ria_reader, ria_reader_with_date_approx
import tqdm
from models.bottleneck_encoder_decoder import BottleneckEncoderDecoderModel
from transformers import BertTokenizer, EncoderDecoderModel, EncoderDecoderConfig, \
    BertConfig, PreTrainedModel, PretrainedConfig, \
    Trainer, TrainingArguments, logging

import pandas as pd
import numpy as np
import csv
import random

In [2]:
from custom_datasets.mlm_ft_dataset import MLMFTDataset
from transformers import (
    DataCollatorForLanguageModeling,
    AutoModelForMaskedLM,
    Trainer,
    TrainingArguments,
    logging,
)

import torch

In [5]:
lenta_path = '/home/aobuhtijarov/datasets/lenta/lenta-ru-news.train.csv'
ria_path = '/home/aobuhtijarov/datasets/ria/ria.shuffled.train.json'
model_path = '/home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/'

In [6]:
logging.set_verbosity_info()

In [7]:
tokenizer = BertTokenizer.from_pretrained(model_path, do_lower_case=False, do_basic_tokenize=False)

Didn't find file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/added_tokens.json. We won't load it.
Didn't find file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/special_tokens_map.json. We won't load it.
Didn't find file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/tokenizer_config.json. We won't load it.
Didn't find file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/tokenizer.json. We won't load it.
loading file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/vocab.txt
loading file None
loading file None
loading file None
loading file None


In [8]:
model = AutoModelForMaskedLM.from_pretrained(model_path)

loading configuration file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.5.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 119547
}

loading weights file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/pytorch_model.bin
Some weights of the model checkpoint at /home/aobuhtijarov/models/rubert

In [9]:
model.resize_token_embeddings(len(tokenizer))

Embedding(119547, 768, padding_idx=0)

In [10]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm_probability=0.15,
)

In [11]:

lenta_records = []
lenta_records.extend(
    [r for r in tqdm.tqdm(lenta_reader('/home/aobuhtijarov/datasets/lenta/lenta-ru-news.val.csv'))]
)

ria_records = []
ria_records.extend(
    [r for r in tqdm.tqdm(ria_reader_with_date_approx('/home/aobuhtijarov/datasets/ria/ria.shuffled.test.json'))]
)

lenta_records = [r for r in lenta_records if r['date'][:4] in ['2010', '2011', '2012', '2013', '2014']]



75971it [00:05, 13063.02it/s]
43171it [01:15, 573.11it/s] 


In [12]:
full_dataset = MLMFTDataset([t['text'] for t in lenta_records + ria_records], tokenizer)

In [13]:
train_fraq = 0.9

train_size = int(train_fraq * len(full_dataset))
test_size = int((1-train_fraq) * 0.5 * len(full_dataset))

train_dataset, test_dataset, eval_dataset = \
    torch.utils.data.random_split(full_dataset, [train_size, test_size, len(full_dataset) - train_size - test_size])


In [14]:
training_args = TrainingArguments(
    output_dir='./bert_ft',
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=16,
    evaluation_strategy='steps',
    learning_rate=3e-5,
    warmup_steps=300,
    overwrite_output_dir=False,
    logging_steps=25,
    eval_steps=50,
    save_steps=50,
    max_steps=1000,
    save_total_limit=1,
    weight_decay=0.01,
    report_to='wandb',
)

PyTorch: setting up devices


In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 58092
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 16
  Total optimization steps = 1000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: leshanbog (use `wandb login --relogin` to force relogin)


Step,Training Loss,Validation Loss


In [2]:
lenta_path = '/home/aobuhtijarov/datasets/lenta/lenta-ru-news.train.csv'
ria_path = '/home/aobuhtijarov/datasets/ria/ria.shuffled.train.json'
clust_model = '/home/aobuhtijarov/models/lenta_ria_clustering_model/checkpoint-6000/'
tokenizer_path = '/home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/'

In [3]:
lenta_records = [r for r in tqdm.tqdm(lenta_reader(lenta_path))]
ria_records = [r for r in tqdm.tqdm(ria_reader_with_date_approx(ria_path))]
lenta_records = [r for r in lenta_records if r['date'][:4] in ['2010', '2011', '2012', '2013', '2014']]

597175it [00:18, 31701.95it/s]
780385it [09:06, 1428.63it/s]


In [4]:
model = EncoderDecoderModel.from_pretrained(clust_model)
tokenizer = BertTokenizer.from_pretrained(tokenizer_path, do_lower_case=False, do_basic_tokenize=False)
setattr(tokenizer, 'max_tokens_text', 250)

In [5]:
model.cuda();

In [6]:
from utils.clustering_utils import get_text_to_vector_func

In [7]:
def get_embeds_for_records(records, text_to_vector_func):
    embeds = np.zeros((len(records), 768))

    for i in tqdm.trange(len(records)):
        text = records[i]["title"] + ' ' + records[i]["text"]
        text = text.lower().replace('\xa0', ' ').strip()
        embeds[i] = text_to_vector_func(text).detach().cpu().numpy().ravel()
        
    return embeds

In [8]:
text_to_vector_func = get_text_to_vector_func('bert-FirstCLS', model, tokenizer)

lenta_embeds = get_embeds_for_records(lenta_records, text_to_vector_func)
ria_embeds = get_embeds_for_records(ria_records, text_to_vector_func)

 14%|█▍        | 23965/173173 [05:54<36:45, 67.64it/s]


KeyboardInterrupt: 

In [9]:
from sklearn.metrics.pairwise import cosine_distances

In [10]:
import multiprocessing as mp

In [11]:
lenta_embeds = np.random.rand(1000, 768)
ria_embeds = np.random.rand(1000, 768)

In [12]:
pairs = mp.Queue()

In [13]:
def f(inds):
    for i in inds:
        dist = cosine_distances(lenta_embeds[i].reshape(1, -1), ria_embeds).ravel()

        top_ria_inds = np.argsort(dist)[:3]
        lenta_month = lenta_records[i]['date'][5:7]
        lenta_day = lenta_records[i]['date'][8:10]

        for j in top_ria_inds:
            ria_month = ria_records[j]['date'][5:7]
            ria_day = ria_records[j]['date'][8:10]

            if ria_month == lenta_month and ria_day == lenta_day:
                pairs.put((i, j))
                break

In [16]:
works = [range(n, len(lenta_embeds), 40) for n in range(40)]
with mp.Pool(processes=40) as pool:
    pool.map(f, works)

In [29]:
pairs = []

for i in tqdm.trange(len(lenta_embeds)):
    dist = cosine_distances(lenta_embeds[i].reshape(1, -1), ria_embeds).ravel()
    
    top_ria_inds = np.argsort(dist)[:3]
    lenta_month = lenta_records[i]['date'][5:7]
    lenta_day = lenta_records[i]['date'][8:10]
    for j in top_ria_inds:
        ria_month = ria_records[j]['date'][5:7]
        ria_day = ria_records[j]['date'][8:10]
        
        if ria_month == lenta_month and ria_day == lenta_day and j not in taken_ria:
            pairs.append((i, j))
            taken_ria.add(j)
            break

  0%|          | 170/173173 [12:58<220:10:15,  4.58s/it]


KeyboardInterrupt: 

In [40]:
x = pairs[random.randint(0, len(pairs)-1)]

In [41]:
lenta_records[x[0]]

{'text': 'президент сша барак обама извинился перед молодоженами, которым из-за него пришлось перенести свадебную церемонию. администрация главы государства запретила устраивать торжество на поле для гольфа, расположенном на военной базе на гавайях перед домом, где президент проводит рождественские каникулы. свадьбу пришлось провести неподалеку от дома командира военной базы, передает cnn. как рассказала сестра жениха, военнослужащие натали хеймель (natalie heimel) и эдвард маллью-младший (edward mallue jr.) пригласили обаму на свою свадьбу 28 декабря. президент прислал ответ, в котором сообщалось, что он не сможет прийти на торжество. за сутки до церемонии паре позвонили и сказали, что праздник нельзя проводить на поле для гольфа. "невесте было очень тяжело. за 24 часа до торжества ей пришлось менять все, что она запланировала", — заявила сестра жениха. после того как агентство bloomberg попросило администрацию президента прокомментировать сложившуюся ситуацию, президент позвонил неве

In [42]:
ria_records[x[1]]

{'text': 'москва, 30 дек — риа новости. президент сша барак обама принес извинения молодоженам за то, что им пришлось перенести церемонию бракосочетания, причиной чему он косвенно был, сообщает cbs news. капитан эдвард малу и его невеста натали хеймел заранее забронировали место проведения свадебной церемонии в гольф-клубе на гавайских островах. перед этим пара отправила бараку обаме приглашение на свадьбу, зная, что президент с семьей проводит в гольф-клубе рождественский отпуск. глава государства вежливо отклонил приглашение, а затем за сутки до торжества молодых уведомили, что им придется перенести место проведения церемонии в связи с тем, что президент решил сыграть в гольф со своими старыми школьными друзьями. сообщается, что управляющий гольф-клубом, не уведомив администрацию белого дома, от своего лица сообщил военнослужащим о необходимости изменить свои свадебные планы в связи с работой спецслужб по обеспечению безопасности президента. узнав о случившемся, барак обама в телефон

In [22]:
len(lenta_records), len(ria_records)

(173173, 780385)

'москва, 23 дек — рапси, мария петрова. европейский суд по пр'

In [119]:
bad_shit = []

for x in ria_records:
    if 'риа новости' not in x['text'][:60]:
        bad_shit.append(x)

In [201]:
def prepend(x):
    x = str(x)
    if len(x) == 2:
        return x
    else:
        return '0' + x

In [120]:
len(bad_shit) / len(ria_records), len(ria_records) - len(bad_shit)

(0.13360955165760108, 744005)

In [198]:
months = [' янв', ' фев', ' мар', ' апр', ' мая', ' июн', ' июл', ' авг', ' сен', ' окт', ' ноя', ' дек']
dates = [str(i) for i in range(1, 32)]

In [1]:
def ria_date_from_text(text):
    try:
        a = text[:70].split('риа новости')[0].split(', ')[1]

        date = '2010-'

        for i, m in enumerate(months):
            if m in a:
                date += prepend(i + 1) + '-'
                date += prepend(a.split(m)[0])

        assert len(date) == 10
        return date  
    except:
        return ''

In [240]:
n=random.randint(0, len(ria_records))
text = ria_records[n]['text']
print(text)
ria_date_from_text(text)

три автомобиля столкнулись на улице новая башиловка на участке между метро савеловская и динамо. пострадавших нет. легковой автомобиль одного из участников дтп получил серьезные повреждения передней части.


IndexError: list index out of range

In [14]:
val_lenta_records = [r for r in tqdm.tqdm(lenta_reader('/home/aobuhtijarov/datasets/lenta/lenta-ru-news.val.csv'))]

75971it [00:02, 36925.84it/s]


75971

In [15]:
sum(1 for r in val_lenta_records if r['date'][:4] in ['2010', '2011', '2012', '2013', '2014'])

21376

In [5]:
len(lenta_records), len(ria_records)

(597175, 891612)

In [9]:
random.shuffle(ria_records)

In [12]:
val_ria_records = [r for r in tqdm.tqdm(ria_reader('/home/aobuhtijarov/datasets/ria/ria.shuffled.val.json'))]

49261it [00:27, 1792.26it/s]


In [13]:
len(val_ria_records)

49261

In [21]:
x = [2,1]
x.extend([1, 5])
x

[2, 1, 1, 5]

In [3]:
cd ../../datasets/

/home/aobuhtijarov/datasets


In [19]:
400000 * 0.05

20000.0

In [3]:
import json
import random
import copy
import torch
import tqdm

In [4]:
from _jsonnet import evaluate_file as jsonnet_evaluate_file

In [5]:
config = json.loads(jsonnet_evaluate_file('../configs/gen_title.jsonnet'))

tokenizer_model_path = '/Users/leshanbog/Documents/neural_models/rubert_cased_L-12_H-768_A-12_pt/'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_path, do_lower_case=False)

loading configuration file /Users/leshanbog/Documents/neural_models/rubert_cased_L-12_H-768_A-12_pt/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "type_vocab_size": 2,
  "vocab_size": 119547
}

Model name '/Users/leshanbog/Documents/neural_models/rubert_cased_L-12_H-768_A-12_pt/' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-

In [6]:
eval_model_file = '/Users/leshanbog/Documents/neural_models/checkpoint-7000/'
enable_bottleneck = True

In [7]:
test_records = [r for r in ria_reader('/Users/leshanbog/Documents/dataset/ria/ria.shuffled.test.json') if random.random() <= 0.005]

In [8]:
test_dataset = GenTitleDataset(
    test_records,
    tokenizer,
    max_tokens_text=196,
    max_tokens_title=48
)

In [9]:
cls = BottleneckEncoderDecoderModel if enable_bottleneck else EncoderDecoderModel
model = cls.from_pretrained(eval_model_file)
model.eval()

loading configuration file /Users/leshanbog/Documents/neural_models/checkpoint-7000/config.json
Model config EncoderDecoderConfig {
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "/data/aobuhtijarov/models/pretrained_dec_6_layers",
    "add_cross_attention": true,
    "architectures": [
      "BertModel"
    ],
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "decoder_start_token_id": null,
    "directionality": "bidi",
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "eos_token_id": null,
    "finetuning_task": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0",
      "1": "LABEL_1"
    },
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "is_decoder": true,
    "is_encoder_decoder": false,
    "lab

BottleneckEncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, ele

In [10]:
batch_size = 6

In [11]:
for i in tqdm.trange(0, len(test_dataset), batch_size):
    data = test_dataset[i]
    for k in tuple(data.keys()):
        if k not in ('input_ids', 'attention_mask'):
            del data[k]
        else:
            data[k] = data[k].unsqueeze(0)

    for j in range(i + 1, min(i + batch_size, len(test_dataset))):
        for k in data.keys():
            data[k] = torch.cat((data[k], test_dataset[j][k].unsqueeze(0)), dim=0)


    output_ids = model.generate(**data, decoder_start_token_id=model.config.decoder.pad_token_id)
    preds = [
        tokenizer.decode(x[1: torch.max(torch.nonzero(x)).item()]) for x in output_ids
    ]
    
    break

  0%|          | 0/40 [00:01<?, ?it/s]


ValueError: You have to specify either input_ids or inputs_embeds

In [26]:
txt = 'как дела у тебя дружочек братишка?'

inp2 = tokenizer(
            txt,
            add_special_tokens=True,
            max_length=196,
            padding="max_length",
            truncation=True
        )

inp = dict()
inp['input_ids'] = torch.LongTensor(inp2['input_ids']).unsqueeze(0)
inp['attention_mask'] = torch.LongTensor(inp2['attention_mask']).unsqueeze(0)

In [27]:
model.generate(**inp, decoder_start_token_id=model.config.decoder.pad_token_id)

ValueError: You have to specify either input_ids or inputs_embeds

In [1]:
from sklearn.cluster import AgglomerativeClustering
from _jsonnet import evaluate_file as jsonnet_evaluate_file
import json
from transformers import EncoderDecoderModel, logging, BertTokenizer

from models.bottleneck_encoder_decoder import BottleneckEncoderDecoderModel
from utils.clustering_utils import calc_clustering_metrics, get_text_to_vector_func

logging.set_verbosity_info()

In [2]:
config = json.loads(jsonnet_evaluate_file('../configs/gen_title.jsonnet'))

max_tokens_text = config.pop('max_tokens_text')
max_tokens_title = config.pop('max_tokens_title')
text_to_vec_func = 'bert-MeanSum'

In [3]:
model_path = '/home/aobuhtijarov/models/tg_gen_title_1500/checkpoint-6000/'
gold_markup_file = '/home/aobuhtijarov/datasets/telegram_news/ru_clustering_0517.tsv'
clustering_data_file = '/home/aobuhtijarov/datasets/telegram_news/ru_clustering_data.jsonl'
tokenizer_model_path = '/home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt'

In [4]:
model = EncoderDecoderModel.from_pretrained(model_path)
model.eval()
model.cuda()

loading configuration file /home/aobuhtijarov/models/tg_gen_title_1500/checkpoint-6000/config.json
Model config EncoderDecoderConfig {
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "/home/aobuhtijarov/models/pretrained_dec_6_layers",
    "add_cross_attention": true,
    "architectures": [
      "BertModel"
    ],
    "attention_probs_dropout_prob": 0.2,
    "bad_words_ids": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "decoder_start_token_id": null,
    "directionality": "bidi",
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.2,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0",
      "1": "LABEL_1"
    },
    "initializer_ra

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [5]:
from utils.clustering_utils import *
import numpy as np

In [6]:
tokenizer = BertTokenizer.from_pretrained(tokenizer_model_path, do_lower_case=False, do_basic_tokenize=False)

Didn't find file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/added_tokens.json. We won't load it.
Didn't find file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/special_tokens_map.json. We won't load it.
Didn't find file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/tokenizer_config.json. We won't load it.
Didn't find file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/tokenizer.json. We won't load it.
loading file /home/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/vocab.txt
loading file None
loading file None
loading file None
loading file None


In [7]:
gold_markup = get_gold_markup(gold_markup_file)

url2record, filename2url = get_data_to_cluster(clustering_data_file)
setattr(tokenizer, 'max_tokens_text', max_tokens_text)
text_to_vector_func = get_text_to_vector_func(text_to_vec_func, model, tokenizer)

print('Calculating embeddings...')
embeds = np.zeros((len(url2record.items()), 768))

Calculating embeddings...


In [8]:
import json
import tqdm
from sklearn.cluster import AgglomerativeClustering

In [9]:
total_articles = len(url2record.items())

for i, (url, record) in tqdm.tqdm(enumerate(url2record.items()), total=total_articles):
    text = record["title"] + ' ' + record["text"]
    text = text.lower().replace('\xa0', ' ').strip()
    embeds[i] = text_to_vector_func(text).detach().cpu().numpy().ravel()

100%|██████████| 10730/10730 [02:49<00:00, 63.41it/s]


In [ ]:
clustering_model = AgglomerativeClustering(
        n_clusters=None,
        distance_threshold=0.13,
        linkage="single",
        affinity="cosine"
    )

clustering_model.fit(embeds)
labels = clustering_model.labels_

In [ ]:
id2url = dict()
for i, (url, _) in enumerate(url2record.items()):
    id2url[i] = url

url2label = dict()
for i, label in enumerate(labels):
    url2label[id2url[i]] = label

In [ ]:
not_found_count = 0
for first_url, second_url in list(gold_markup.keys()):
    not_found_in_labels = first_url not in url2label or second_url not in url2label
    not_found_in_records = first_url not in url2record or second_url not in url2record
    if not_found_in_labels or not_found_in_records:
        not_found_count += 1
        print(first_url, second_url)
        gold_markup.pop((first_url, second_url))

In [ ]:
not_found_count